In [1]:
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient
import os, datetime
from dotenv import load_dotenv
from PIL import Image
import io
from tensorflow.keras.preprocessing.image import img_to_array, load_img
import numpy as np

load_dotenv()

def upload_file_to_blob(blob_service_client, container_name, file_path, blob_name):
    try:
        # Create the container if it doesn't already exist
        container_client = blob_service_client.get_container_client(container_name)
        if not container_client.exists():
            container_client = blob_service_client.create_container(container_name)

        # Create a blob client using the local file name as the name for the blob
        blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob_name)

        # Upload the file
        with open(file_path, "rb") as data:
            blob_client.upload_blob(data, overwrite=True)
        print("File was uploaded successfully to Blob storage.")

    except Exception as e:
        print(f"An error occurred: {e}")

def get_blob_service_client(connection_string):
    return BlobServiceClient.from_connection_string(connection_string)

connection_string = os.getenv("CSTR__BLOB__VISINSP")

# Create the BlobServiceClient object which will be used to create a container client
blob_service_client = get_blob_service_client(connection_string)

container_name = "image-uploads"


In [ ]:
if 1 == 2:
    file_path = r"C:\Users\oliver.koehn\Documents\gitProjects\qualityInspection\.gitignore"
    blob_name = "dummytest123"
    upload_file_to_blob(blob_service_client, container_name, file_path, blob_name)

In [ ]:
base_path_local_files = r"C:\Users\oliver.koehn\Downloads\qualityInsepction\casting_data\casting_data\test\ok_front"
l__ok_images = [10,14,16,31]
for n in l__ok_images:
    fn_original = f"cast_ok_0_{n}.jpeg"
    fn_blob = fn_original.replace(".jpeg", f"__{datetime.datetime.now().strftime('%Y%m%d%H%M%S%f')}.jpeg")
    fp_local = os.path.join(base_path_local_files, fn_original)
    fp_blob = f"{datetime.datetime.now().strftime('%Y')}/{datetime.datetime.now().strftime('%m')}/{datetime.datetime.now().strftime('%d')}/{fn_blob}"
    print(f"UPLOAD LOCAL FILE {fn_original} to BLOB AS {fn_blob}")
    print(f"fp_local: {fp_local}")
    print(f"fp_blob: {fp_blob}")
    upload_file_to_blob(blob_service_client, container_name, fp_local, fp_blob)

In [21]:
def read_image_from_blob(blob_service_client, container_name, blob):
    blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob)
    stream = blob_client.download_blob()
    return Image.open(io.BytesIO(stream.readall())) # image.show()

def format_image_for_model(image):
    image = image.convert('RGB')
    image = image.resize((200, 200))
    image_array = img_to_array(image)
    image_array = image_array * (1./255)
    return np.expand_dims(image_array, axis=0)

def eval_img_from_blob(loaded_model, blob_service_client, container_name, blob):
    image = read_image_from_blob(blob_service_client, container_name, blob)
    img_array = format_image_for_model(image)
    result = loaded_model.predict(img_array)
    prob__ok = result[0][1]
    #j__db = {"FP__BLOB" : blob, "CONTAINER_NAME" : container_name, "PROBABILITY_OK": prob__ok}

    return prob__ok

eval_img_from_blob(loaded_model, blob_service_client, container_name, "2024/05/21/cast_ok_0_31__20240521155649383512.jpeg")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step


0.99995387